In [ ]:
# This code is to download historical daily weather data from
# weatherunderground.com
# This code will require station_list.csv
# Please setup paths to run the code succesfully

# AUTHOR: Haowei Wu

In [1]:
# code is modified from http://learn.devfe.st/datascience/


days_per_month = {1: 31, 2: 28, 3: 31, 4: 30,
                  5: 31, 6: 30, 7: 31, 8: 31,
                  9: 30, 10: 31, 11: 30, 12: 31}

link_format = "http://www.wunderground.com/history/airport/{}/{}/{}/{}/DailyHistory.html"

import os
import csv
import requests
import os.path
import pandas as pd
from bs4 import BeautifulSoup

def download_file(link, name):
    if os.path.isfile(name):
        return
    file = open(name, 'w')
    r = requests.get(link)
    file.write(r.text)
    file.close()

fields = ['Mean Temperature', 'Max Temperature', 'Min Temperature',\
          'Dew Point', 'Average Humidity', 'Maximum Humidity',\
          'Minimum Humidity', 'Precipitation', 'Wind Speed',\
          'Max Wind Speed', 'Max Gust Speed']

def test_station(name):
    with open(name) as fin:
        soup = BeautifulSoup(fin.read(), "html.parser")
    test = soup.find(id="historyTable").find_all("tr")
    return isnull(test)

def scrape_file(name):
    with open(name) as fin:
        soup = BeautifulSoup(fin.read(), "html.parser")
    data = {}
    for row in soup.find(id="historyTable").find_all("tr"):
        cells = row.find_all("td")
        if len(cells) == 4:
            name = cells[0].text.strip()
            if name in fields:
                if len(cells[1].text.split()) == 0:
                    data[name] = "-"
                else:
                    data[name] = cells[1].text.split()[0].strip()   # Split to remove units
    return data

def test_station(name):
    with open(name) as fin:
        soup = BeautifulSoup(fin.read(), "html.parser")
    test = soup.find(id="historyTable")
    return test != None


def scandirs(path):
    for root, dirs, files in os.walk(path):
        for currentFile in files:
            #print ("processing file: " + currentFile)
            exts = ('.html')
            if any(currentFile.lower().endswith(ext) for ext in exts):
                os.remove(os.path.join(root, currentFile))
                


In [6]:
stations = pd.read_csv("/Users/Haowei_Wu/Big_data_analytics_project/station_list.csv")

# select the stations, years you want to download
for station in range(1431,1450):
    city = stations.at[station,'Station']
    state = stations.at[station,'State']
    code = stations.at[station,'airportCode']
    links = [link_format.format(code, year, month, day)
             for year in range(2006, 2016) # 2013 - 2015 inclusive
             for month in range(1, 13)     # 1 - 12 inclusive
             for day in range(1, days_per_month[month] + 1)]
    for i, link in enumerate(links):
        #if i % 50 == 0:
            #print("Done with %d.." % i)
        download_file(link, "%d.html" % i)
        
    csv_fields = ["Month", "Day", "Year"] + fields
    with open(code+"_"+city+"_"+state+"_weather_data.csv", "w") as fout:
        writer = csv.DictWriter(fout, csv_fields)
        writer.writeheader()

        for i, link in enumerate(links):
            test = test_station("{}.html".format(i))
            if test:
                data = scrape_file("{}.html".format(i))
                url_parts = link.split("/")
                data["Month"] = int(url_parts[-3])
                data["Year"] = int(url_parts[-4])
                data["Day"] = int(url_parts[-2])
                writer.writerow(data)
    scandirs("/Users/Haowei_Wu/Big_data_analytics_project/bda_proj_1400_1450")
    print ('finish ',station/stations.shape[0]*100, "%")
    

finish  89.32584269662921 %
finish  89.38826466916355 %
finish  89.45068664169787 %
finish  89.51310861423221 %
finish  89.57553058676653 %
finish  89.63795255930087 %
finish  89.70037453183521 %
finish  89.76279650436953 %
finish  89.82521847690387 %
finish  89.8876404494382 %
finish  89.95006242197253 %
finish  90.01248439450687 %
finish  90.0749063670412 %
finish  90.13732833957553 %
finish  90.19975031210986 %
finish  90.2621722846442 %
finish  90.32459425717853 %
finish  90.38701622971286 %
finish  90.4494382022472 %
